In [22]:
!pip install selenium







In [23]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options




In [24]:

options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'



In [25]:
pip install webdriver_manager


Note: you may need to restart the kernel to use updated packages.


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager
import time



# LinkedIn credentials
linkedin_email = "chetnagirish2@gmail.com"
linkedin_password = "Boss1234$"

# Set up GeckoDriver
GeckoDriverManager().install()

'C:\\Users\\chetn\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [27]:
# Initialize the Firefox webdriver
options = Options()
driver = webdriver.Firefox(options=options)

In [28]:
# Open LinkedIn login page
driver.get('https://www.linkedin.com/jobs')

In [29]:
email_input = driver.find_element("name", "session_key")
password_input = driver.find_element("name", "session_password")
login_button = driver.find_element("xpath", "//button[@type='submit']")

In [30]:
email_input.send_keys(linkedin_email)
password_input.send_keys(linkedin_password)

In [31]:
login_button.click()